In [5]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

In [6]:
df = pd.read_csv('data/postings.csv')

In [7]:
df.head()

,title,body,bullets
0,"Data Scientist - Mountain View, CA","Data Scientist - Mountain View, CA\nGroundTrut...","('Help senior members of the team to explore, ..."
1,"Data Scientist - Seattle, WA","Data Scientist - Seattle, WA\nA Bachelor or Ma...",('A Bachelor or Masters Degree in a highly qua...
2,"Junior Data Scientist - College Park, MD 20740","Junior Data Scientist - College Park, MD 20740...",('Degree: Bachelor’s degree in business analyt...
3,"Data Scientist - New York, NY","Data Scientist - New York, NY\nDescription\nDS...","('Languages: Python, PySpark, SQL', 'Data Tool..."
4,"(Entry-Level) Data Scientist - Chicago, IL","(Entry-Level) Data Scientist - Chicago, IL\nDa...",('Be the go-to person for Data ingest and stor...


In [31]:
vectorizer = CountVectorizer(stop_words='english')

In [32]:
tf_matrix = vectorizer.fit_transform(df.bullets)

In [33]:
tf_np_matrix = tf_matrix.toarray()

In [34]:
tf_vector = tf_np_matrix[0]

In [35]:
non_zero_indices = np.flatnonzero(tf_vector)

In [36]:
num_unique_words = non_zero_indices.size

In [37]:
num_unique_words

67

In [38]:
non_zero_indices

array([ 253,  283,  468,  575,  718,  925,  958, 1010, 1062, 1333, 1343,
       1345, 1392, 1409, 1464, 1493, 1619, 1843, 1844, 1955, 1994, 2020,
       2078, 2168, 2436, 2481, 2748, 3036, 3040, 3074, 3146, 3197, 3229,
       3265, 3281, 3338, 3433, 3710, 3838, 3866, 4026, 4072, 4089, 4121,
       4153, 4324, 4332, 4412, 4469, 4651, 4790, 4897, 4902, 4903, 4964,
       4991, 4995, 5041, 5076, 5143, 5153, 5203, 5253, 5275, 5478, 5811,
       5966])

In [39]:
words = vectorizer.get_feature_names()

In [40]:
unique_words = [words[i] for i in non_zero_indices]

In [41]:
unique_words

['algorithms',
 'amazon',
 'athena',
 'basis',
 'build',
 'closely',
 'collaborate',
 'communication',
 'computer',
 'daily',
 'data',
 'databases',
 'deep',
 'degree',
 'derive',
 'develop',
 'dive',
 'engineering',
 'engineers',
 'excellent',
 'experience',
 'explore',
 'familiarity',
 'fluency',
 'hadoop',
 'help',
 'insights',
 'learn',
 'learning',
 'libraries',
 'location',
 'machine',
 'managers',
 'master',
 'mathematics',
 'members',
 'mllib',
 'nosql',
 'open',
 'optimize',
 'peer',
 'petabytes',
 'phd',
 'pipelines',
 'plus',
 'product',
 'productionize',
 'prototypes',
 'python',
 'relational',
 'rich',
 'science',
 'scientists',
 'scikit',
 'senior',
 'services',
 'set',
 'significant',
 'skills',
 'source',
 'spark',
 'sql',
 'statistics',
 'stores',
 'team',
 'use',
 'web']

In [42]:
posting_1 = {'Word': unique_words, 
             'Count': tf_vector[non_zero_indices]}

posting_df = pd.DataFrame(posting_1).sort_values('Count', ascending=False)

In [43]:
posting_df.head(n=10)

,Word,Count
10,data,4
31,machine,3
20,experience,3
28,learning,3
21,explore,2
44,plus,2
1,amazon,2
62,statistics,2
60,spark,2
45,product,2
